In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)

from compute_ET_green import compute_et_green, calculate_band_std_dev

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)
from utils import date_utils, ee_utils

from typing import List, Tuple

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

## Compute and export ET Green


In [5]:
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"
YEARS = range(2022, 2023)

In [6]:
DOUBLE_CROPPING_IMAGE = ee.Image(
    "projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022"
)

In [7]:
# path_to_WaPOR_ET_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor"


# ET_COLLECTIONs = ee_utils.merge_collections(
#     years=YEARS, asset_name=path_to_WaPOR_ET_asset
# )

# ET_COLLECTION = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Zuerich/ET_WaPOR_10m_dekadal_2022"
# ).map(lambda img: back_to_float(img, 100))

ET_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_30m_ZH_2018-2022"
).map(lambda img: back_to_float(img, 100))


# Replace negative pixel values with 0 while retaining image properties
def set_negative_to_zero(img):
    return img.max(0).copyProperties(img, img.propertyNames())


ET_COLLECTION = ET_COLLECTION.map(set_negative_to_zero)

ee_utils.print_value_ranges(ET_COLLECTION, "fitted_ET")
date_utils.print_collection_dates(ET_COLLECTION)

Image 1: Min = 0.00, Max = 23.35
Image 2: Min = 0.00, Max = 32.00
Image 3: Min = 0.00, Max = 59.53
Image 4: Min = 0.00, Max = 91.67
Image 5: Min = 0.00, Max = 121.08
Image 6: Min = 0.00, Max = 138.73
Image 7: Min = 0.00, Max = 141.40
Image 8: Min = 0.00, Max = 127.94
Image 9: Min = 0.00, Max = 102.29
Image 10: Min = 0.00, Max = 68.56
Image 11: Min = 0.00, Max = 39.24
Image 12: Min = 0.00, Max = 20.12
Image 13: Min = 0.00, Max = 17.11
Image 14: Min = 0.00, Max = 28.31
Image 15: Min = 0.00, Max = 55.86
Image 16: Min = 0.00, Max = 89.48
Image 17: Min = 0.00, Max = 119.90
Image 18: Min = 0.00, Max = 137.09
Image 19: Min = 0.00, Max = 138.93
Image 20: Min = 0.00, Max = 125.23
Image 21: Min = 0.00, Max = 100.73
Image 22: Min = 0.00, Max = 66.93
Image 23: Min = 0.00, Max = 35.84
Image 24: Min = 0.00, Max = 16.59
Image 25: Min = 0.00, Max = 13.88
Image 26: Min = 0.00, Max = 26.97
Image 27: Min = 0.00, Max = 56.03
Image 28: Min = 0.00, Max = 89.39
Image 29: Min = 0.00, Max = 119.51
Image 30: Mi

In [8]:
ET_COLLECTION.first().bandNames().getInfo()

['fitted_ET']

In [9]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [10]:
def process_and_export_et_green(
    et_collection_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
    double_cropping_image: ee.Image = None,
) -> List[ee.batch.Task]:
    """
    Process and export ET green images for a given year.

    Args:
        et_collection (ee.List): List of ET images
        year (int): Year to process
        aoi (ee.Geometry): Area of interest
        time_steps (int): Number of time steps (36 for dekadal, 12 for monthly)
        time_step_type (str): Type of time step ("dekadal" or "monthly")
        double_cropping_image (ee.Image): Double cropping image

    Returns:
        List[ee.batch.Task]: List of export tasks
    """
    nutzung_collection = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_{year}"
    )

    # Uncomment this if not working with Zuerich!
    nutzung_collection = nutzung_collection.map(
        lambda feature: feature.set("nutzung", feature.get("NUTZUNG"))
    )

    jurisdictions = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Zuerich/zurich_jurisdictions_{year}_wgs84"
    )

    double_cropping = ee.Image(double_cropping_image)

    # Prepare crop filters
    not_irrigated_crops = get_crops_to_exclude()
    rainfed_crops = get_rainfed_reference_crops()
    exclude_filter, rainfed_filter = create_crop_filters(
        not_irrigated_crops, rainfed_crops
    )

    # Process nutzung collection
    nutzung_with_double_crop = add_double_cropping_info(
        nutzung_collection, double_cropping
    )
    _, rainfed_fields = filter_crops(
        nutzung_with_double_crop, exclude_filter, rainfed_filter
    )

    tasks = []
    for i in range(time_steps):
        # Get time step info
        if time_step_type == "dekadal":
            dekadal = i % 3 + 1
            month = i // 3 + 1
            time_step_name = f"{month:02d}_D{dekadal}"
        elif time_step_type == "monthly":
            month = i + 1
            time_step_name = f"{month:02d}"
        else:
            raise ValueError("time_step_type must be either 'dekadal' or 'monthly'")

        # Get ET image and compute ET green
        et_image = ee.Image(et_collection_list.get(i))
        et_green = compute_et_green(
            et_image, rainfed_fields, jurisdictions, et_band_name="fitted_ET"
        )

        et_green = back_to_int(et_green, 100)

        # Export result
        task_name = (
            f"ET_green_landsat_30m_jurisdiction_{time_step_type}_ZH_{year}-{time_step_name}"
        )
        asset_id = f"projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_30m_jurisdiction_{time_step_type}_ZH_2022/{task_name}"

        task = export_image_to_asset(et_green, asset_id, task_name, year, aoi, scale=30)
        tasks.append(task)

    return tasks

In [11]:
# for year in YEARS:

#     et_collection_list = ET_COLLECTION.filterDate(
#         f"{year}-01-01", f"{year}-12-31"
#     ).toList(TIME_STEPS)

#     # DOUBLE_CROPPING_IMAGE = ee.Image(f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}")

#     tasks = process_and_export_et_green(
#         et_collection_list, year, AOI, TIME_STEPS, TIME_STEP_TYPE, DOUBLE_CROPPING_IMAGE
#     )

#     print(f"Started {len(tasks)} export tasks.")

In [12]:
# collection.size().getInfo()

# date_utils.print_collection_dates(collection)

### Sanity check


In [13]:
check_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_30m_jurisdiction_monthly_ZH_2022"
).map(lambda img: back_to_float(img, 100))

check_collection.first().bandNames().getInfo()

['ET_green']

In [14]:
date_utils.print_collection_dates(check_collection)
print_value_ranges(check_collection, "ET_green")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = 0.12, Max = 6.36
Image 2: Min = 5.25, Max = 17.26
Image 3: Min = 19.77, Max = 41.51
Image 4: Min = 38.16, Max = 71.45
Image 5: Min = 56.09, Max = 99.88
Image 6: Min = 67.75, Max = 116.75
Image 7: Min = 70.63, Max = 118.48
Image 8: Min = 63.42, Max = 103.79
Image 9: Min = 46.55, Max = 77.76
Image 10: Min = 25.74, Max = 45.91
Image 11: Min = 8.36, Max = 21.10
Image 12: Min = 0.15, Max = 8.22


In [16]:
Map = geemap.Map()

image = ee.Image(check_collection.toList(check_collection.size()).get(6))

vis_params = {
    "bands": ["ET_green"],
    "min": 70,
    "max": 100,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

Map.center_object(AOI, 12)
Map.addLayer(image, vis_params, "ET_green")

Map

Map(center=[47.41511720684401, 8.655992127451274], controls=(WidgetControl(options=['position', 'transparent_b…